## Exploratory Data Analysis with Pandas

The notebook is an example project on exploratory data analysis (EDA).  The purpose of EDA is to explore a given dataset, see what possible hypotheses about which data variables can be formulated, which models can be built, and if more data needs to be collected. Thus, EDA is  carried out as a step before building any particular statistical model or testing a particular hypothesis.

This EDA project will work with the Titanic dataset we saw in the previous session. Download the file "titanic.xlsx" from Blackboard, and place it in the same folder as this notebook.

In this notebook, we will use Pandas tools to obtain answers to different questions about the data, by examining different characteristics of the dataset and its selected parts, creating  visualizations and running basic statistical tests.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_excel("titanic.xlsx")

Once the data is loaded into a dataframe, we can examine it. Because the dataset is quite large, let's look at the first 3 rows:

In [ ]:
# Print the first 3 rows
df.head(3)

We can also examine the last 3 rows:

In [ ]:
# the last 3 rows
df.tail(3)

So each row is a record on one passenger, consisting of various data about him or her. We can also see how the data is encoded. For example, whether the passenger survived or not is encoded as 1 (survived) or 0 (did not survive). We also see that there are missing values in some columns, for some passengers (see the `NaN` values).

How many rows and columns are there?

In [ ]:
df.shape # (n rows, n cols)

So there are data on 1309 passengers, represented in 14 columns.

What are the columns and their data types?

In [ ]:
df.dtypes

* "sibsp" stands for the number of siblings or spouses on board for a passenger;
* "parch" stands for the number of children or parents on board;
* "embarked" is the port where the passenger embarked on the ship;
* "home.dest" is the port of destination for the passenger.

The names of the other columns are self-explanatory.

Let's view the descriptive statistics on the numerical columns.

In [ ]:
df.describe()

We can immediately make a number of observations, for example: 
* the mean value of the "survived" column is 0.38, which means 38% of the passengers survived in this trip;
* the quartile figures suggest that at least 50% of the passengers travelled in the third passenger class;
* 75% of the passengers paid less than $31.27 for the trip;
* the youngest passenger was just several months old, the oldest 80 years old.

We can also visualize the distribution of the values in some columns. For example, let's see how the fare values are distributed:

In [ ]:
df.fare.hist(bins=15)

The plot shows that the vast majority of the passengers paid under \$33 for the trip, but there were a few who paid over \$200.

Survival factors
========

Let's find out how many people survived. We can use the `value_count` method of a Series, which outputs the number of unique values in a given column.

In [ ]:
# How many people survived?

# value_counts counts unique values in a column
df.survived.value_counts()

500 people survived, 809 died. We can see the percent of the unique values in a column, by setting `normalize=True`:

In [ ]:
# How many people survived, in percent?
df.survived.value_counts(normalize=True)

38% survived.

The `crosstab` function in Pandas allows to compare two columns in a dataframe, displaying a **contingency table**. We can use it, for example, to find out how many men and women survived and how many died:

In [ ]:
# How many men and women survived and how many died?
pd.crosstab(df.survived, df.sex)

So here, the contingency table has two rows (the two possible values of "survived") and two columns (the two possible values of "sex"). Each cell encodes the number of records where one unique value in one column coincides with one unique value in the other. For example, the top left cell shows how many women died - there are 127.

As with `value_counts`, `cross_tab` can display the same as percentage:

In [ ]:
# Percent of women among survivors? percent of men among survivors?
pd.crosstab(df.survived, df.sex, normalize="index")

The `normalize` argument is set to "index", which means the percentage is calculated in rows. Thus, the tables shows that among the victims 15.6% were women, 84.3% were men. Among the survivors, 67% were women and 32% were men.

We can also calculate percentage in columns, setting `normalize="columns"`:

In [ ]:
# Percent of survivors among women? percent of survivors among men?
pd.crosstab(df.survived, df.sex, normalize="columns")

This table shows that, of all women 27.2% died, 72.7% survived. Of all men, 80.9% died and 19% survived.

We can also select a subset of observations based on some criteria and return their number. For example, we can select all passengers under the age of 5 and return their count, using the `len` function:

In [ ]:
# How many children under 5 on board?
len(df[df.age <= 5])

There were 56 children under the age of 5.

We can further examine this particular subset of observations, selecting specific columns and using the same methods as above. For example, let's find out how many children under the age of 5 survived?

In [ ]:
# How many of them survived?

# subset of observations where the age is <= 5
children_df = df[df.age <= 5]

# the series with surviving children
sur_children = children_df["survived"]

# unique values of in this series
sur_children.value_counts()

That is, 37 children under the age of 5 survived, 19 died.

The three lines of code above could be "chained", as follows, to produce the same result:

In [ ]:
df[df.age <= 5]["survived"].value_counts()

In other words, we perform exactly the same set of steps as before, but without storing intermediate results in separate variables.

Another way to answer the same question is to select observations, based on the complex criteria (age <=5 and survived == 1) and just return the count of these observations:

In [ ]:
# Alternatively:
len(df[(df.age <= 5) & (df.survived == 1)])

The same numbers, in percentage:

In [ ]:
# How many children under 5 survived, in percent?
df[df.age <= 5]["survived"].value_counts(normalize=True)

The "sibsp" column contains information on how many siblings or spouses a passenger had on board of the ship. Let's find out if having a spouse or sibling on-board increase survival chances for a passenger.

First, let's see what are unique values in this column.

In [ ]:
# Did having a spouse or sibling on-board increase survival chances?

# What are possible values of sibsp?
df['sibsp'].value_counts()

So most of the passengers (891 out of 1309) were travelling on their own. Quite a few had a sibling or a spouse with them, but also there were several large families. For example, 9 passengers had 8 siblings travelling with them, which seems like a family of 9 brothers and sisters.

Let's see if the average number of siblings or spouses per survivor is greater than the average number of siblings per non-survivor. If it is, then it would mean that having a sibling or a spouse on board increased one's survival chances.

First, we need to retrieve observations where "survival" is 1, look up the "sibsp" column for them and find out the mean number of its values:

In [ ]:
# What is the average number of siblings or spouses per survivor?
df[df['survived'] == 1]['sibsp'].mean()

A survivor has 0.46 siblings or spouses, on average.

How about non-survivors?

In [ ]:
# What is the average number of siblings or spouses for non-survivors?
df[df['survived'] == 0]['sibsp'].mean()

Interestingly, people who died had on average more siblings or spouses on-board.

Let's see if the difference between the two mean is statistically significant. We will use the independent samples t-test, implemented in the "scipy" package (which is also part of Anaconda, and you should have it already installed):

In [ ]:
# Is the difference significant?
# independent t-test
from scipy.stats import ttest_ind

# create two series: 
# one is the "sibsp" column of those entries where "survived" is 1
series1 = df[df['survived'] == 1]['sibsp']
# and the other is the "sibsp" column of those entries where "survived" is 0
series2 = df[df['survived'] == 0]['sibsp']

# ttest_ind expects two NumPy arrays as input, 
# so we need to input the `values` of the two series
tval, pval = ttest_ind(series1.values, series2.values)

# ttest_ind returns the t-value and p-value
print("T test: %.5f, at p=%.5f" % (tval, pval))

The p-value is greater than 0.05 (the usual significance level), so we cannot reject the null hypothesis that there is no difference between the two samples. In other words, having a spouse or a sibling on board did not have any significant effect on passengers' survival chances.

We will learn more about testing the significance of the difference between samples in the next unit.

Points of Embarkation
=====================

Let's find out where the passengers boarded the ship. We can view unique values of the "embarked" column:

In [ ]:
# What are all the points of embarkation?
df.embarked.unique()

There are four possible values: "S" (Southampton, England), "C" (Cherbourg, France), "Q" (Queens, Ireland), and `NaN` for those passengers whose point of embarkation is unknown.

How many people embarked at each point? 

In [ ]:
# use "dropna=False" to print also the number of those whose point of embarkation is unknown.
df.embarked.value_counts(dropna=False)

The majority got on the ship at Southampton, and only for two passengers the point of embarkation is not known.

Destinations
======

Similarly, let's find out where the passengers were headed. There are lots of unique destinations, so let's print only 10 of them:

In [ ]:
# What were the 10 most common final destinations?

df["home.dest"].value_counts().head(10)

The "home.dest" column looks like free-text values, i.e., the entries are not standardized. We can try to extract certain parts of information from the column in order to standardize it a little. For example, we can extract the names of US states and Canadian provinces, because they follow a similar pattern (two capital letters like "NY"). We'll then be able to find out which state or province is the most popular destination.

In [ ]:
# What are the most common states which were the final destinations?

# create a new column with the destination state
df['state.dest'] = df['home.dest'].str.extract("([A-Z][A-Z])", expand=False)
df.head(3)

So here, we've used a regular expression to extract sequences of two capital letters from the "home.dest" column, and then saved the result to a new column, "state.dest".

Now, we can find out which is the most common state or province among destinations). We will again use `value_counts`, but because it drops `NaN` values by default, we need to specify that we don't want them to be dropped, in order to see for how many passengers the destination is unknown:

In [ ]:
# What are the 10 most common values in the state.dest column?
df['state.dest'].value_counts(dropna=False).head(10)

So it looks like for many people we could not extract the state/province information, using the regex. This suggests we should probably improve the regex. But along those observtions where the state was extracted, we see that New York (NY), Pennsylvania (PA), and New Jersey (NJ) are the most common destinations.

We can use a regex to try and find people who were headed for Canada. The regex may be "Canada|ON|PQ|BC", i.e., we are interested in observations where in the "home.dest" we can find either "Canada", "ON" (Ontario), "PQ" (Province of Quebec), or "BC" (British Columbia).

Let's create a temporary dataframe, from which we drop all entries that have `NaN` for "home.dest", because presence of `NaN` cause an error to be thrown when calling the `contains` method below.

In [ ]:
# How many people were headed for Canada?

# create a temporary df so that we can drop some rows from it
df2 = df.copy()
df2 = df2.dropna(subset=["home.dest"])

Then, in the temporary dataframe, we can select observations that contain the regex, and output the count of the observations using `len`:

In [ ]:
# How many people were heading to Canada?
len(df2[df2["home.dest"].str.contains("Canada|ON|PQ|BC")])

The regex is most likely not perfect, but it covers the most common Canadian destinations and can give us a rough estimate for how many people were travelling to Canada.

Age and passenger class
===============

Now, let's examine the data on the age and the passenger class.

First, let's see if we have the age information on every passenger.

In [ ]:
# How many people were there whose age is unknown?
len(df[df.age.isnull()])

Actually, for 263 people the age is unknown.

How about the passenger class?

In [ ]:
len(df[df.pclass.isnull()])

The passenger class is known for every one of them. So let's drop all observations, where the age is unknown.

In [ ]:
# Clean the data: drop all records where either age or pclass is undefined
df = df.dropna(subset=['age'])

How many records are now left?

In [ ]:
df.shape

What is the mean age in each passenger class?

We first group the observations by the passenger class using `.groupby` and then access the "age" column in the resulting dataframe, and then calculate the mean in each group:

In [ ]:
df.groupby('pclass')['age'].mean()

The mean age in the first class is 39.1, in the second 29.5, in the third 24.8. That is, younger people tended to travel in the cheapest class.

Now let's visualize the distribution of the first-class passengers by the age. 

We first select the relevant set of observations (i.e., where "pclass" is 1), take the "age" column in them, and plot a histogram (`.hist`) with 10 bins:

In [ ]:
# histogram of ages of passengers in 1st class
df[df.pclass==1]['age'].hist(bins=10)

The distribution resembles the normal one: the most common bin is the one which contains the mean age of the passengers in the first class.

A similar histogram for the third-class passengers:

In [ ]:
# histogram of ages of passengers in 3rd class
df[df.pclass==3]['age'].hist(bins=10)

We can create multi-level groups of observations, by supplying a list of column names to `.groupby`. For example, we can first group passengers by their class, and then within each passenger class we can group the passengers by their gender. 

Then we can inspect each group. In this way, we can answer questions like "What is the mean age of different sexes in each passenger class?".

In [ ]:
# What was the mean age of different sexes in each passenger class?
df.groupby(['pclass', 'sex'])['age'].mean()

So it appears that, for example, the mean age of women in the third class was just 22, and in each class the mean age of men was slightly greater than the mean age of women.

We can use `.groupby` to find out the percentage of survivors in each passenger class. Because 1 in the "survived" column stands for "survived" and 0 for otherwise, the mean will be the percentage of survivors:

In [ ]:
# How many survivors in each class, percent-wise?
df.groupby('pclass').survived.value_counts(normalize=True)

Thus, in the first class, the survival rate was 63%, in the second 44%, and in the third only 26%.